# Распознавание букв Backpropagation
Чуть изменённая задача из книги Роберта Каллана 2001 гг. "Основные концепции нейронных сетей". стр. 63 "Классификация символов"

Реализация в Numpy (успех зависит от нач иниц)

Классификация символов A, B, C, D. Чёрно-белые изображения букв 8 на 8, 6 картинок на букву (обучение - по 4, 2 тестовые)

Сойдёт использование 2-х сигмоид и выходные векторы как 00 для A, 01 для B и т.д. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image
path = "D:/pseudo_data/"
a = []; b = []; c = []; d = []
n_imgperl = 6
for i in range(n_imgperl):
    a.append(Image.open(path + f"a{i}.png"))
    b.append(Image.open(path + f"b{i}.png"))
    c.append(Image.open(path + f"c{i}.png"))
    d.append(Image.open(path + f"d{i}.png"))

In [2]:
def PixelArray(img):
    img = np.array(img)
    oz = np.zeros((8, 8))
    for i in range(img.shape[0]):
        oz[i] = [x[0] for x in img[i]]
    oz[oz == 0] = 1
    oz[oz > 1] = 0 # 255 - это белый, белый пусть будет 0
    return oz

In [3]:
letter0 = np.array([PixelArray(a[0]), PixelArray(b[0]), PixelArray(c[0]), PixelArray(d[0])])
letter1 = np.array([PixelArray(a[1]), PixelArray(b[1]), PixelArray(c[1]), PixelArray(d[1])])
letter2 = np.array([PixelArray(a[2]), PixelArray(b[2]), PixelArray(c[2]), PixelArray(d[2])])
letter3 = np.array([PixelArray(a[3]), PixelArray(b[3]), PixelArray(c[3]), PixelArray(d[3])])
letter4 = np.array([PixelArray(a[4]), PixelArray(b[4]), PixelArray(c[4]), PixelArray(d[4])])
letter5 = np.array([PixelArray(a[5]), PixelArray(b[5]), PixelArray(c[5]), PixelArray(d[5])])
answers = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#answers = np.zeros((4, 4))
#for i in range(4):
#    answers[i, i] = 1
#X = np.concatenate((letter1, letter2, letter3, letter4, letter5))
X = np.concatenate((letter0, letter1, letter2, letter3)) ; Y = np.concatenate((answers, answers, answers, answers))
#Y = np.concatenate((answers, answers, answers, answers, answers))

In [4]:
def net(x, w):
    return np.dot(x, w)
def sigmoid(wx):
    return 1 / (1 + np.exp(-wx))
def sigderiv(sig_neuron):
    return sig_neuron * (1 - sig_neuron)

In [5]:
def forward_pass(x, w_hid, w_out):
    neurons = sigmoid(net(x, w_hid))
    output = sigmoid(net(neurons, w_out))
    return neurons, output
def predict(x, w_hid, w_out):
    x = np.append(x.reshape(1, -1), [[-1]])
    neurons = sigmoid(net(x, w_hid))
    output = sigmoid(net(neurons, w_out))
    return output
    #return np.round(output)

In [6]:
def Learning(epsilon, X, Y, num_hid, lmd):
    inp_size = np.size(X[0]) + 1 ; out = Y[0].size
    w_hid = np.random.uniform(-1, 1, (inp_size, num_hid))
    w_out = np.random.uniform(-1, 1, (num_hid, out))
    max_error = float('inf') 
    epoch = 0
    while (max_error > epsilon):
        max_error = epsilon * 1.1
        for i in range(X.shape[0]):
            x = np.append(X[i], [[1]])
            x = x.reshape(1, -1) #shape: (1, 64) (65 т.к. фиктивный признак 1)
            y = Y[i]
            neurons, output = forward_pass(x, w_hid, w_out)
            error = y - output
            delta = error * sigderiv(output) # (1, 2) локальный градиент выходного нейрона
            delta2 = sigderiv(neurons) * np.dot(delta, np.transpose(w_out)) #лок град скрытых
            w_out += lmd * np.dot(np.transpose(neurons), delta)
            w_hid += lmd * np.dot(np.transpose(x), delta2) 
            #print(output, y)
            if abs(max_error) < abs(error.any()):
                max_error = np.sum(abs(error))
        if epoch % 1000 == 0:
            print(f'\t{epoch} {max_error}')
        epoch += 1
    return w_hid, w_out, epoch

In [7]:
hidden_weights, output_weights, number_of_epochs = Learning(1.5e-3, X, Y, 8, 9e-1)

	0 1.2053139284804133
	1000 0.015786299817998728
	2000 0.010914699200446298
	3000 0.008825239915279726
	4000 0.007599077256464581
	5000 0.006770486071440485
	6000 0.0061630595007408084
	7000 0.00569342083206148
	8000 0.005316408310756526
	9000 0.005005167269653277
	10000 0.00474260199267218
	11000 0.004517246689483945
	12000 0.004321087772854559
	13000 0.004148333959776762
	14000 0.003994682479606992
	15000 0.003856861086663943
	16000 0.0037323312240967743
	17000 0.0036190893682158665
	18000 0.003515530360501229
	19000 0.0034203511006039022
	20000 0.0033324812348829447
	21000 0.003251032333478565
	22000 0.0031752599985681584
	23000 0.00310453518835474
	24000 0.0030383222207527316
	25000 0.0029761616931218704
	26000 0.002917657070619195
	27000 0.0028624640471832663
	28000 0.002810282026457167
	29000 0.002760847241011999
	30000 0.0027139271502080176
	31000 0.0026693158451362997
	32000 0.002626830253517176
	33000 0.002586306985058573
	34000 0.0025475996933805867
	35000 0.00251057685745204

In [8]:
print(f"На обучение было потрачено {number_of_epochs} эпох.")
for i in range(letter1.shape[0]):
    #print(f'\t{[round(i) for i in predict(letter3[i], hidden_weights, output_weights)]} {answers[i]}')
    print(f'\t{predict(letter4[i], hidden_weights, output_weights)} {answers[i]}')

На обучение было потрачено 97379 эпох.
	[0.00014914 0.00044558] [0 0]
	[5.85469870e-04 9.98446777e-01] [0 1]
	[0.91646838 0.05339177] [1 0]
	[0.99885154 0.99921665] [1 1]


In [9]:
print(f"На обучение было потрачено {number_of_epochs} эпох.")
for i in range(letter2.shape[0]): #letter1
    #x,o=forward_pass(np.append(X[i], [[-1]]).reshape(1, -1))
    #print(x, o)
    #print(f'\t{[round(i) for i in predict(letter3[i], hidden_weights, output_weights)]} {answers[i]}')
    print(f'\t{predict(letter5[i], hidden_weights, output_weights)} {answers[i]}')
    #print(X[i])

На обучение было потрачено 97379 эпох.
	[0.00033333 0.00095498] [0 0]
	[0.00102148 0.99889626] [0 1]
	[0.99940307 0.00268753] [1 0]
	[0.98519241 0.99939279] [1 1]


In [10]:
print(f'\t{predict(PixelArray(Image.open(path + f"d99.png")), hidden_weights, output_weights)}')

	[0.9969708  0.99943064]
